In [ ]:
import pandas as pd

# データフレームの全列の文字列表示幅を設定
pd.set_option('display.max_colwidth', None)

# Parquetファイルを読み込む
exceptions_df = pd.read_parquet('../00_Extract_Exceptions/exceptions.parquet')
exceptions_df

In [3]:
import os
import shutil

def get_user_choice(prompt, valid_choices):
    """ユーザーに選択肢を提示し、有効な入力を受け取る"""
    while True:
        choice = input(f"{prompt} ({'/'.join(valid_choices)}): ").strip().lower()
        if choice in valid_choices:
            return choice
        print("Invalid choice. Please try again.")

def create_unique_folder(folder_path):
    """フォルダを作成し、上書きするか連番を付けるかを選択"""
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        return folder_path

    choice = get_user_choice(f"Folder '{folder_path}' already exists. Overwrite?", ["yes", "no", "skip"])
    if choice == "yes":
        return folder_path
    elif choice == "no":
        # 連番を付けて新しいフォルダを作成
        base_folder = folder_path
        counter = 1
        while os.path.exists(folder_path):
            folder_path = f"{base_folder}_{counter}"
            counter += 1
        os.makedirs(folder_path)
        return folder_path
    elif choice in ["skip", "cancel"]:
        return None

def move_file_with_prompt(src_path, dest_path):
    """ファイルを移動し、上書きするか連番を付けるかを選択"""
    if not os.path.exists(dest_path):
        shutil.move(src_path, dest_path)
        return

    choice = get_user_choice(f"File '{dest_path}' already exists. Overwrite?", ["yes", "no", "skip"])
    if choice == "yes":
        shutil.move(src_path, dest_path)
    elif choice == "no":
        # 連番を付けて新しいファイル名で移動
        base_dest = os.path.splitext(dest_path)[0]
        ext = os.path.splitext(dest_path)[1]
        counter = 1
        while os.path.exists(dest_path):
            dest_path = f"{base_dest}_{counter}{ext}"
            counter += 1
        shutil.move(src_path, dest_path)
    elif choice in ["skip", "cancel"]:
        print(f"Skipped moving file: {src_path}")

def create_folders_and_move_files(exception_df):
    for path in exception_df['path']:
        # ファイルのディレクトリとファイル名を分割
        file_dir = os.path.dirname(path)
        file_name = os.path.basename(path)
        
        # ファイル名から拡張子を除去して新しいフォルダ名を作成
        file_name_without_ext = os.path.splitext(file_name)[0]
        new_folder_name = f"{file_name_without_ext}_created"
        new_folder_path = os.path.join(file_dir, new_folder_name)
        
        # 新しいフォルダを作成（選択肢を提示）
        new_folder_path = create_unique_folder(new_folder_path)
        if new_folder_path is None:
            print(f"Skipped creating folder for file: {path}")
            continue
        
        # ファイルを新しいフォルダに移動（選択肢を提示）
        dest_file_path = os.path.join(new_folder_path, file_name)
        move_file_with_prompt(path, dest_file_path)

# 実行
create_folders_and_move_files(exceptions_df)

In [ ]:
# 例外ファイルの再確認
import os
import yaml

def get_files_in_secondary_folders(config_path):
    # 設定ファイルを読み込む
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    
    # 設定からディレクトリリスト、無視する拡張子とファイル名を取得
    secondary_folders = config.get('secondary_folders', [])
    ignore_extensions = config.get('ignore_extensions', [])
    ignore_files = config.get('ignore_files', [])
    
    # 各フォルダ内のファイルを収集
    collected_files = []
    for folder in secondary_folders:
        if not os.path.isdir(folder):
            print(f"Warning: '{folder}' is not a valid directory.")
            continue
        
        # ディレクトリ直下のファイルを取得
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            
            # ファイル名が無視リストに含まれているか、拡張子が無視対象かを確認
            if os.path.isfile(file_path):
                ext = os.path.splitext(file_name)[1].lower()  # 拡張子を小文字で取得
                if file_name in ignore_files or ext in ignore_extensions:
                    continue  # 無視対象の場合スキップ
                
                collected_files.append(file_path)
    
    # データフレーム形式に変換して返す
    return pd.DataFrame(collected_files, columns=['path'])

# 設定ファイルのパス
config_path = '../00_Extract_Exceptions/config.yaml'

# ファイルパスを取得し、データフレームとして保存
files_df = get_files_in_secondary_folders(config_path)

# 結果を出力（必要に応じて表示）
files_df

In [11]:
# 確認で出てきたファイルでも同じようにフォルダを作る
create_folders_and_move_files(files_df)

In [16]:
# 患者フォルダ一覧の作成

def get_directories_in_secondary_folders(config_path):
    # 設定ファイルを読み込む
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    
    # 設定からディレクトリリストを取得
    secondary_folders = config.get('secondary_folders', [])
    
    # ベースフォルダを設定（'mydata/00_working' へのパス）
    base_folder = os.path.normpath('../mydata/00_working')
    
    # secondary_folders直下のフォルダを収集
    collected_dirs = []
    for folder in secondary_folders:
        if not os.path.isdir(folder):
            print(f"Warning: '{folder}' is not a valid directory.")
            continue
        
        # フォルダ直下のディレクトリを取得
        for item in os.listdir(folder):
            item_path = os.path.join(folder, item)
            if os.path.isdir(item_path):  # ディレクトリの場合のみ収集
                # 'mydata/00_working' からの相対パスを計算
                relative_path = os.path.relpath(item_path, base_folder)
                collected_dirs.append({
                    "dirname": item,
                    "path": relative_path
                })
    
    # データフレーム形式に変換して返す
    df = pd.DataFrame(collected_dirs)
    df.index.name = 'index'
    return df.reset_index()

# 設定ファイルのパス
config_path = '../00_Extract_Exceptions/config.yaml'

# ディレクトリ一覧を取得し、データフレームとして保存
directories_df = get_directories_in_secondary_folders(config_path)

# データフレームをParquetとして保存
output_parquet_path = '../01_Process_Exceptions/patients_dirs.parquet'
directories_df.to_parquet(output_parquet_path, index=False)
print(f"Directory list saved to {output_parquet_path}")

Directory list saved to ../01_Process_Exceptions/patients_dirs.parquet


In [19]:
len(directories_df)

803

In [20]:
# すべての拡張子を確認する

def get_all_extensions(directories_df, base_folder='../mydata/00_working'):
    """
    患者フォルダ内の全ファイルを走査して、ユニークな拡張子を抽出します。

    Parameters:
        directories_df (pd.DataFrame): 患者フォルダ一覧（path列が相対パス）
        base_folder (str): 患者フォルダのベースディレクトリ（相対パス計算の基準）

    Returns:
        list: ユニークな拡張子のリスト
    """
    extensions = set()  # 重複を避けるためにセットを使用

    # 各患者フォルダを処理
    for _, row in directories_df.iterrows():
        folder_path = os.path.join(base_folder, row['path'])  # フルパスを構築
        if not os.path.isdir(folder_path):
            print(f"Warning: '{folder_path}' is not a valid directory.")
            continue

        # フォルダ内のすべてのファイルを取得
        for root, _, files in os.walk(folder_path):
            for file in files:
                ext = os.path.splitext(file)[1].lower()  # 拡張子を小文字で取得
                if ext:  # 拡張子が空でない場合
                    extensions.add(ext)

    # ソートしてリストに変換して返す
    return sorted(list(extensions))

# ベースフォルダのパス
base_folder = '../mydata/00_working'

# 患者フォルダ内の拡張子を収集
unique_extensions = get_all_extensions(directories_df, base_folder)

# 結果を表示
print("Unique extensions found:")
print(unique_extensions)

Unique extensions found:
['.ai', '.cr2', '.dat', '.db', '.doc', '.docx', '.ini', '.jpg', '.json', '.key', '.pdf', '.png', '.ppt', '.pptx', '.pspimage', '.tga', '.tif', '.tmp', '.xls', '.xlsm']
